# **5.1 Exploring Quantum Chemistry with GDB1k**

In [1]:
!pip install --pre deepchem

In [2]:
import deepchem as dc
import warnings
warnings.filterwarnings('ignore')
dc.__version__

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


'2.8.1.dev'

In [3]:
import deepchem as dc
from sklearn.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge

In [4]:
tasks = ["atomization_energy"]
dataset_file = "../../datasets/gdb1k.sdf"
smiles_field = "smiles"
mol_field = "mol"

In [5]:
featurizer = dc.feat.CoulombMatrixEig(23, remove_hydrogens=False)

Atomization Energy : 분자를 구성하는 모든 원자 사이의 결합을 끊어 각각의 기체 상태 원자로 만드는데 필요한 에너지 - 화학적 안정성 척도

Coulomb Matrix : 전하량비례 거리 반비례 3D정전기적 구조를 숫자로 바꿈

Smiles와 다르게 원자의 실제 3차원 위치 정보 포함, 양자역학적 에너지 계산에 유리

Zero-padding : 분자마자 원자수가 다르므로 최대 원자수를 정해놓고 남는 자리는 0으로 채워 행렬 크기 맞춤

In [8]:
import os
import requests

# Check if the file exists locally, if not, download it
file_path = '/content/gdb1k.sdf'
if not os.path.exists(file_path):
    print("Downloading gdb1k.sdf...")
    url = "https://raw.githubusercontent.com/deepchem/deepchem/master/deepchem/data/gdb1k.sdf"
    r = requests.get(url, allow_redirects=True)
    with open(file_path, 'wb') as f:
        f.write(r.content)
    print("Download complete.")

# Update dataset_file to the correct path
dataset_file = file_path

loader = dc.data.SDFLoader(
      tasks=tasks,
      featurizer=featurizer)
dataset = loader.create_dataset(dataset_file)

Download complete.


In [9]:
random_splitter = dc.splits.RandomSplitter()
train_dataset, valid_dataset, test_dataset = random_splitter.train_valid_test_split(dataset)

In [ ]:
transformers = [
    dc.trans.NormalizationTransformer(transform_X=True, dataset=train_dataset),
    dc.trans.NormalizationTransformer(transform_y=True, dataset=train_dataset)]

for dataset in [train_dataset, valid_dataset, test_dataset]:
  for transformer in transformers:
      dataset = transformer.transform(dataset)

Normalization : 쿨롱행렬의 값은 원자 번호와 거리에따라 숫자 크기가 매우 달라짐 -> 학습 불안정 or 특정 변수에 과하게 의존 -> Normatlization 평균0 표준편차1로 맞춰줌

In [ ]:
def rf_model_builder(model_dir, **model_params):
  sklearn_model = RandomForestRegressor(**model_params)
  return dc.models.SklearnModel(sklearn_model, model_dir)
params_dict = {
    "n_estimators": [10, 100],
    "max_features": ["auto", "sqrt", "log2", None],
}

metric = dc.metrics.Metric(dc.metrics.mean_absolute_error)
optimizer = dc.hyper.GridHyperparamOpt(rf_model_builder)
best_rf, best_rf_hyperparams, all_rf_results = optimizer.hyperparam_search(
    params_dict, train_dataset, valid_dataset, output_transformers=transformers,
    metric=metric, use_max=False)
for key, value in all_rf_results.items():
    print(f'{key}: {value}')
print('Best hyperparams:', best_rf_hyperparams)

In [ ]:
def krr_model_builder(model_dir, **model_params):
  sklearn_model = KernelRidge(**model_params)
  return dc.models.SklearnModel(sklearn_model, model_dir)

params_dict = {
    "kernel": ["laplacian"],
    "alpha": [0.0001],
    "gamma": [0.0001]
}

metric = dc.metrics.Metric(dc.metrics.mean_absolute_error)
optimizer = dc.hyper.GridHyperparamOpt(krr_model_builder)
best_krr, best_krr_hyperparams, all_krr_results = optimizer.hyperparam_search(
    params_dict, train_dataset, valid_dataset, output_transformers=transformers,
    metric=metric, use_max=False)
for key, value in all_krr_results.items():
    print(f'{key}: {value}')
print('Best hyperparams:', best_krr_hyperparams)

💡 Chapter 5.1 Insight

3차원 정전기적 구조를 2D행렬로 표현하여 원자간 상호작용 모델에 반영 - Coulomb Matrix 사용

입력값과 타겟값의 넓은 범위를 Normalization으로 표준화해 수치적 안정성 확보